As part of next submission, I will be training RandomForest Regressor.

I will be using data from submission-1 where the nulls for outpatients are replaced by 0.

I will be using data from submission-2 where the nulls for outpatients are replaced by other logic explained in that submission.

In [49]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
with open('healthcare-claims-prediction_kaggel/patient_data_train.json', 'r') as file:
    train_data_json = json.load(file)
with open('healthcare-claims-prediction_kaggel/patient_data_test.json', 'r') as file:
    test_data_json = json.load(file)
train_claims = pd.read_csv('healthcare-claims-prediction_kaggel/train.csv')

def normalize_and_clean_data(data):
    '''
    Normalizing and handling the data
    '''
    import warnings
    warnings.filterwarnings("ignore")
    df = pd.json_normalize(data)

    # each patient is unique record,i will be replacing nulls with 0 values

    # so First we will do this for the features related to past years
    df["Out patient costs.2023"].fillna(0,inplace=True)

    df["Out patient costs.2022"].fillna(0,inplace=True)

    df["Out patient costs.2021"].fillna(0,inplace=True)

    df["Out patient costs.2020"].fillna(0,inplace=True)

    df["Out patient costs.2019"].fillna(0,inplace=True)

    df["Conditions.HD"].fillna(0,inplace=True)

    df["Conditions.HT"].fillna(0,inplace=True)

    df["Conditions.DB"].fillna(0,inplace=True)

    df["Conditions.AT"].fillna(0,inplace=True)

    return df

train_data = normalize_and_clean_data(train_data_json)
test_data = normalize_and_clean_data(test_data_json)
    # As we already know, we have only 5 past years to handle the values
    # Now, we need to do the similar kind of analysis for conditions features as well.

train_data = pd.merge(train_data, train_claims, on='PatientID')

trian_ID_col = train_data[["PatientID"]]
train_data.drop(["PatientID"],axis=1,inplace=True)
test_ID_col = test_data[["PatientID"]]
test_data.drop(["PatientID"], axis=1,inplace=True)


from sklearn.preprocessing import LabelEncoder

sc = LabelEncoder()
sc.fit(train_data["Sex"])
train_data["Sex"] = sc.transform(train_data["Sex"])
test_data["Sex"] = sc.transform(test_data["Sex"])

x = train_data.drop(["TotalClaims"],axis=1)
y = train_data[["TotalClaims"]]

# These lines are making both the features in simlar order

temp = test_data[["Conditions.AT","Conditions.DB"]]
test_data.drop(["Conditions.AT","Conditions.DB"], axis=1, inplace=True)
test_data["Conditions.DB"] = temp["Conditions.DB"]
test_data["Conditions.AT"] = temp["Conditions.AT"]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)

# Sclaing the data

from sklearn.preprocessing import StandardScaler

sc= StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)
test_data = sc.transform(test_data)



In [51]:
# Random Forest Hyperparameter Grid
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rf_param_grid = {
     'n_estimators': [100, 200, 300, 500, 1000],
    'max_depth': [None, 10, 20, 30, 15],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True, False]
}

rf = RandomForestRegressor(random_state=42)

rf_random_search_before = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=rf_param_grid, 
    n_iter=10,  # Number of parameter settings to try
    cv=5,  # 5-fold cross-validation
    verbose=2, 
    random_state=42, 
    n_jobs=-1  # Use all available processors
)

rf_random_search_before.fit(x_train, y_train)
print(f"Best parameters for Random Forest: {rf_random_search_before.best_params_}")


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for Random Forest: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 15, 'bootstrap': True}


In [52]:

from sklearn.metrics import mean_absolute_error
rf_best_before = rf_random_search_before.best_estimator_
y_train_pred = rf_best_before.predict(x_train)
y_pred_rf = rf_best_before.predict(x_test)
print("Random Forest performance:")

print("Train Accuracy: ",mean_absolute_error(y_train, y_train_pred))
print("Test Accuracy: ", mean_absolute_error(y_test, y_pred_rf))



Random Forest performance:
Train Accuracy:  1180.138191239635
Test Accuracy:  1931.3677290945993


Data from submission-2 where the nulls have been replaced with mean of previous values.

In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json


In [55]:
with open('healthcare-claims-prediction_kaggel/patient_data_train.json', 'r') as file:
    train_data_json = json.load(file)
with open('healthcare-claims-prediction_kaggel/patient_data_test.json', 'r') as file:
    test_data_json = json.load(file)

train_claims = pd.read_csv('healthcare-claims-prediction_kaggel/train.csv')

def normalize_and_clean_data(data):
    '''
    Normalizing and handling the data
    '''
    import warnings
    warnings.filterwarnings("ignore")
    df = pd.json_normalize(data)

    # each patient is unique record,i will be replacing nulls with 0 values

    # so First we will do this for the features related to past years

    df["Conditions.HD"].fillna(0,inplace=True)

    df["Conditions.HT"].fillna(0,inplace=True)

    df["Conditions.DB"].fillna(0,inplace=True)

    df["Conditions.AT"].fillna(0,inplace=True)

    return df

train_data = normalize_and_clean_data(train_data_json)
test_data = normalize_and_clean_data(test_data_json)
    # As we already know, we have only 5 past years to handle the values
    # Now, we need to do the similar kind of analysis for conditions features as well.



train_data = pd.merge(train_data, train_claims, on='PatientID')

def fill_na_respectively(row, x, mean_val, feature_1, feature_2):
    
    if pd.isna(row[feature_2]):
        if pd.isna(row[feature_1]):
            row[feature_2] = mean_val
        else:
            row[feature_2] = row[feature_1] - x
    return row

respective_dif = {}

def find_dif(df, feature_1, feature_2):
    df_filtered = df.dropna(subset=[feature_1, feature_2])
    df_features2 = df.dropna(subset=[feature_2])
    name = feature_1 + feature_2
    df_filtered["difference"] = df_filtered[feature_1] - df_filtered[feature_2]
    respective_dif[name] = df_filtered["difference"].mean()



# Finding difference between 2022 and 2023
feature_1 = "Out patient costs.2023"
feature_2 = "Out patient costs.2022"
find_dif(train_data, feature_1, feature_2)

# Finding difference between 2021 and 2022
feature_1 = "Out patient costs.2022"
feature_2 = "Out patient costs.2021"
find_dif(train_data, feature_1, feature_2)


# Finding difference between 2021 and 2020
feature_1 = "Out patient costs.2021"
feature_2 = "Out patient costs.2020"
find_dif(train_data, feature_1, feature_2)

# Finding difference between 2020 and 2019
feature_1 = "Out patient costs.2020"
feature_2 = "Out patient costs.2019"
find_dif(train_data, feature_1, feature_2)

# print(respective_dif)


# Before filling rest of the cols, I will be filling the 2023 column
mean_values = train_data["Out patient costs.2023"].mean()
train_data["Out patient costs.2023"] = train_data["Out patient costs.2023"].fillna(mean_values)


# Filling null values in 2022 column
feature_1 = "Out patient costs.2023"
feature_2 = "Out patient costs.2022"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
train_data = train_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Filling null values in 2021 column
feature_1 = "Out patient costs.2022"
feature_2 = "Out patient costs.2021"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
train_data = train_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)

# Filling null values in 2020 column
feature_1 = "Out patient costs.2021"
feature_2 = "Out patient costs.2020"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
train_data = train_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Filling null values in 2019 column
feature_1 = "Out patient costs.2020"
feature_2 = "Out patient costs.2019"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
train_data = train_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Repeating the same for the test data

# Before filling rest of the cols, I will be filling the 2023 column
mean_values = train_data["Out patient costs.2023"].mean()
test_data["Out patient costs.2023"] = test_data["Out patient costs.2023"].fillna(mean_values)


# Filling null values in 2022 column
feature_1 = "Out patient costs.2023"
feature_2 = "Out patient costs.2022"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
test_data = test_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Filling null values in 2021 column
feature_1 = "Out patient costs.2022"
feature_2 = "Out patient costs.2021"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
test_data = test_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)

# Filling null values in 2020 column
feature_1 = "Out patient costs.2021"
feature_2 = "Out patient costs.2020"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
test_data = test_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)


# Filling null values in 2019 column
feature_1 = "Out patient costs.2020"
feature_2 = "Out patient costs.2019"
name = feature_1 + feature_2
mean_val = train_data[feature_2].mean()
test_data = test_data.apply(lambda row: fill_na_respectively(row, respective_dif[name], mean_val, feature_1, feature_2), axis=1)

trian_ID_col = train_data[["PatientID"]]
train_data.drop(["PatientID"],axis=1,inplace=True)
test_ID_col = test_data[["PatientID"]]
test_data.drop(["PatientID"], axis=1,inplace=True)


# converting Sex column into quantitative
from sklearn.preprocessing import LabelEncoder

sc = LabelEncoder()
sc.fit(train_data["Sex"])
train_data["Sex"] = sc.transform(train_data["Sex"])
test_data["Sex"] = sc.transform(test_data["Sex"])


x = train_data.drop(["TotalClaims"],axis=1)
y = train_data[["TotalClaims"]]



# These lines are making both the features in simlar order

temp = test_data[["Conditions.AT","Conditions.DB"]]
test_data.drop(["Conditions.AT","Conditions.DB"], axis=1, inplace=True)
test_data["Conditions.DB"] = temp["Conditions.DB"]
test_data["Conditions.AT"] = temp["Conditions.AT"]



from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42)


# Sclaing the data

from sklearn.preprocessing import StandardScaler

sc= StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)
test_data = sc.transform(test_data)




In [56]:
# Random Forest Hyperparameter Grid
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rf_param_grid = {
     'n_estimators': [100, 200, 300, 500, 1000],
    'max_depth': [None, 10, 20, 30, 15],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True, False]
}

rf = RandomForestRegressor(random_state=42)

rf_random_search_after = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=rf_param_grid, 
    n_iter=10,  # Number of parameter settings to try
    cv=5,  # 5-fold cross-validation
    verbose=2, 
    random_state=42, 
    n_jobs=-1  # Use all available processors
)

rf_random_search_after.fit(x_train, y_train)
print(f"Best parameters for Random Forest: {rf_random_search_after.best_params_}")


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best parameters for Random Forest: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 15, 'bootstrap': True}


In [57]:

from sklearn.metrics import mean_absolute_error
rf_best_after = rf_random_search_after.best_estimator_
y_pred_rf = rf_best_after.predict(x_test)
y_train_pred = rf_best_after.predict(x_train)
print("Random Forest Performance:")

print("Train Accuracy: ",mean_absolute_error(y_train, y_train_pred))
print("Test Accuracy: ", mean_absolute_error(y_test, y_pred_rf))



Random Forest Performance:
Train Accuracy:  1206.5699266300846
Test Accuracy:  1978.753094563964


In [58]:
res = rf_best_after.predict(test_data)
test_ID_col["TotalClaims"] = res
test_ID_col.to_csv("submission.csv" , index = False)

In [59]:
res = rf_best_before.predict(test_data)
test_ID_col["TotalClaims"] = res
test_ID_col.to_csv("submission.csv" , index = False)